# Classifiers

In [ ]:
import csv
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
import re 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
import nltk
from nltk.corpus import stopwords

## Preprocessing

In [ ]:
import random
rows=[]
filename=('newbig.csv')
with open(filename, 'r') as csvfile: 
    data = csv.reader(csvfile) 
    for row in data:
        rows.append(row)
    rows.pop(0)
print(len(rows))

print(len(rows))
for i in range(10):
    print(rows[i])

In [ ]:
#preprocesses data by extracting the window of words around the target preposition. We use stemmers and lemmatisers for preprocessing the text.
def remove(list): 
    pattern = '[0-9]'
    list = [re.sub(pattern, '', i) for i in list] 
    return list
temp=[]
sentence_list=[]
Y=[]
X=[]
    
n=len(rows)
for i in range(n):
    print(rows[i])
    words=[]
    l=[]
    l=rows[i][0].split(' ')
    print(l)
    l=remove(l)
    if rows[i][1] == 'p70' or rows[i][1] == '9' or rows[i][1] == 'MASCOT':
        continue
    Y.append(rows[i][2])
    k=l.index(rows[i][1])   
    l_limit=k-5
    u_limit=k+5
    if l_limit<=0:
        l_limit=0

    if u_limit>len(l):
        u_limit=len(l)

    words=l[l_limit:u_limit+1]
    words.remove(l[k])
    sw = stopwords.words('english')
    for i in words:
        for j in sw:
            if i == j:
                words.remove(i)
    new_words=[]
    for i in words:
        stemmer=SnowballStemmer('english')
        k=stemmer.stem(i)
        new_words.append(k)
    # print(new_words)
    final_string=''
    for i in new_words:
        final_string=final_string+' '+i
   
       sentence_list.append(final_string)

print(len(sentence_list))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
matrix=TfidfVectorizer(max_features=3000)
X = matrix.fit_transform(sentence_list).toarray()

X_data=np.array(X)
Y_data=np.array(Y)

print(type(X_data))
print(type(Y_data))
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.1)

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

# Predict Class
y_pred = classifier.predict(X_test)

# Accuracy 
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)

## Random Forests

hyper parameter tuning was first done using randomised search CV and then we used grid search CV to get the o[timal set of hyper parameters by considering values close to the random value predicted by the randomised search CV.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
n_estimators = [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]

# Number of features to consider at every split
max_features = ['auto', 'sqrt', 10, 50, 100]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 20]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 5, 10]
# Method of selecting samples for training each tree
# bootstrap = [True, False]
bootstrap = [True]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:

from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [90,100,150],
    'max_features': [40,50,60],
    'min_samples_leaf': [1],
    'min_samples_split': [4,5,6],
    'n_estimators': [300,400,500]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

grid_search.fit(X_train,y_train)



In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

rfc = RandomForestClassifier(bootstrap =True,max_depth=90,max_features=50,min_samples_leaf= 1,min_samples_split=5,n_estimators=500)
rfc.fit(X_train,y_train)
y_pred = rfc.predict(X_test)
scores = cross_val_score(rfc, X_train, y_train, cv=5)
print(scores)

## SVM 

Since literature suggests to use Grid Search CV for finding the optimal set of hyperparameters for SVM's we have considered a few values and then applied this technique.

In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    kernels=[ 'rbf','linear',]
    param_grid = {'C': Cs, 'kernel': kernels,'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    print(grid_search.best_params_)

svc_param_selection(X_train, y_train,5)


In [ ]:
from sklearn import svm
from sklearn.model_selection import KFold
clf = svm.SVC(kernel='linear', gamma=0.01, C=10) 
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

## Metrics

In [ ]:
conf_mat = [[0,0],[0,0]]
y_pred=y_pred.tolist()
y_test=y_test.tolist()

i=0
print("confusion matrix:")

while i < len(y_pred):
    
    if y_pred[i]=='1' and y_test[i]=='1': 
        conf_mat[1][1] +=1
    elif y_pred[i]=='0' and  y_test[i]=='1': 
        conf_mat[1][0] +=1
    elif y_pred[i]=='1' and  y_test[i]=='0': 
        conf_mat[0][1] +=1
    elif y_pred[i]=='0' and y_test[i]== '0': 
        conf_mat[0][0] +=1
    i=i+1
print(conf_mat)
for list1 in conf_mat:
	print (list1)

print("\n")	


In [ ]:
# print("\n")	
# print("Precision")
if conf_mat[1][1]==0 and conf_mat[1][0]==0:
	p = (float(conf_mat[1][1]))/(float(conf_mat[1][1]) + float(conf_mat[0][1])+1)
else:
	p = (float(conf_mat[1][1]))/(float(conf_mat[1][1]) + float(conf_mat[0][1]))

if conf_mat[1][1]==0 and conf_mat[0][1]==0:
	r = (float(conf_mat[1][1]))/(float(conf_mat[1][1]) + float(conf_mat[1][0]+1))
else:
	r = (float(conf_mat[1][1]))/(float(conf_mat[1][1]) + float(conf_mat[1][0]))


acc= (float(conf_mat[0][0])+float(conf_mat[1][1]))/(float(conf_mat[0][0])+float(conf_mat[0][1])+float(conf_mat[1][0])+float(conf_mat[1][1]))


if p==0 and r==0:
	F1=2*p*r/(p+r+1)
else :
	F1=2*p*r/(p+r)
 
print(p,r,acc,F1)

ZeroDivisionError: ignored